# 1. 메모리 변수 제거

In [1]:
# 메모리 변수 모두 제거
all = [var for var in globals() if var[0] != "_"]
for var in all:
    del globals()[var]

# 2. 사용 패키지

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 3. 데이터 로드

In [3]:
train = pd.read_csv('data/train.csv') # 훈련 데이터
test = pd.read_csv('data/test.csv') # 테스트 데이터
submission = pd.read_csv('data/sample_submission.csv') # 미션 데이터
lunch = pd.read_csv('data/lunch.csv') # 중식 시간대 날씨
dinner = pd.read_csv('data/dinner.csv') # 석식 시간대 날씨
corona = pd.read_csv('data/corona.csv') # 진주시 코로나 신규 확진자
level = pd.read_csv('data/level.csv') # 진주시 코로나 거리두기 단계

# 4. 데이터 설명

## 1) train.csv
- 일자 : 연-월-일 (2016-02-01 ~ 2021-01-26)
- 요일 : 월, 화, 수, 목, 금
- 본사정원수 : 총 직원 수
- 본사휴가자수 : 휴가 중인 인원수
- 본사출장자수 : 본사 소속이나 타지역에 근무 중인 인원 수
- 본사시간외근무명령서승인건수 : 해당 일자에 초과 근무한 인원 수
- 현본사소속재택근무자수 : 재택근무하여 구내식당 이용불가한 인원 수
- 조식메뉴 : 아침 식사 메뉴 (각 재료의 원산지)
- 중식메뉴 : 점심 식사 메뉴 (각 재료의 원산지)
- 석식메뉴 : 저녁 식사 메뉴 (각 재료의 원산지)
- 중식계 : 구내식당에서 점심 식사한 인원 수
- 석식계 : 구내식당에서 저녁 식사한 인원 수

## 2) test.csv
- 일자 : 연-월-일 (2021-01-27 ~ 2021-04-09)
- 요일 : 월, 화, 수, 목, 금
- 본사정원수 : 총 직원 수
- 본사휴가자수 : 휴가 중인 인원수
- 본사출장자수 : 본사 소속이나 타지역에 근무 중인 인원 수
- 본사시간외근무명령서승인건수 : 해당 일자에 초과 근무한 인원 수
- 현본사소속재택근무자수 : 재택근무하여 구내식당 이용불가한 인원 수
- 조식메뉴 : 아침 식사 메뉴 (각 재료의 원산지)
- 중식메뉴 : 점심 식사 메뉴 (각 재료의 원산지)
- 석식메뉴 : 저녁 식사 메뉴 (각 재료의 원산지)

## 3) sample_submission.csv
- 일자 : 연-월-일 (2021-01-27 ~ 2021-04-09)
- 중식계 : 구내식당에서 점심 식사할 인원 수를 예측해야함
- 석식계 : 구내식당에서 저녁 식사할 인원 수를 예측해야함

## 4) lunch.csv
- 날짜 : 연-월-일 (2016-02-01 ~ 2021-04-09)
- 체감온도 : 해당 날짜 11 ~ 13시 평균 체감온도
- 불쾌지수 : 68미만 '낮음', 68이상 75미만 '보통', 75이상 80미만 '높음', 80이상 '매우높음'
- 우산 : 외출할 때 우산이 필요한지 여부

## 5) dinner.csv
- 일자 : 연-월-일 (2016-02-01 ~ 2021-04-09)
- 체감온도 : 해당 날짜 17 ~ 19시 평균 체감온도
- 불쾌지수 : 68미만 '낮음', 68이상 75미만 '보통', 75이상 80미만 '높음', 80이상 '매우높음'
- 우산 : 외출할 때 우산이 필요한지 여부

## 6) corona.csv
- 일자 : 연-월-일 (2020-02-28 ~ 2021-04-09)
- 코로나신규확진자 : 해당 일자의 경상남도 진주시청 브리핑 시간 15:30 이전 신규 확진자 수

## 7) level.csv
- 일자 : 연-월-일 (2016-02-01 ~ 2021-04-09)
- 2016-02-01 ~ 2020-01-26 : 평시
- 2020-01-27 ~ 2020-11-25 : 1단계
- 2020-11-26 ~ 2021-01-10 : 2단계 - 진주시 이통장 연수 집단 감염
- 2021-01-11 ~ 2021-01-24 : 2.5단계 - 진주시 교회 집단 감염
- 2021-01-25 ~ 2021-03-11 : 1.5단계 
- 2021-03-12 ~ 2021-04-09 : 2단계 - 진주시 목욕탕 집단 감염

# 5. 날씨 데이터 - Lunch, Dinner

## 1) lunch.csv와 dinner.csv의 column 이름이 같음

In [4]:
lunch.rename(columns = {'날짜' : '일자', '체감온도' : '체감온도(중식)', '불쾌지수' : '불쾌지수(중식)', '우산' : '우산(중식)'}, inplace = True)
dinner.rename(columns = {'날짜' : '일자', '체감온도' : '체감온도(석식)', '불쾌지수' : '불쾌지수(석식)', '우산' : '우산(석식)'}, inplace = True)

## 2) ['일자']를 기준으로 train.csv + lunch.csv

In [5]:
train = pd.merge(train, lunch, left_on = '일자', right_on = '일자', how = 'left')
test = pd.merge(test, lunch, left_on = '일자', right_on = '일자', how = 'left')

## 3) ['일자']를 기준으로 train.csv + dinner.csv

In [6]:
train = pd.merge(train, dinner, left_on = '일자', right_on = '일자', how = 'left')
test = pd.merge(test, dinner, left_on = '일자', right_on = '일자', how = 'left')

# 6. 코로나 데이터 - Corona

## 1) ['일자']를 기준으로 train.csv + corona.csv

In [7]:
train = pd.merge(train, corona, left_on = '일자', right_on = '일자', how = 'left')
test = pd.merge(test, corona, left_on = '일자', right_on = '일자', how = 'left')

## 2) 경상남도 진주시 코로나 첫 확진자는 2020-02-28에 발생하였으므로 그 이전 일자는 결측치

In [8]:
train = train.fillna(0)
test = test.fillna(0)

# 7. 거리두기 데이터 - Level
- 코로나 데이터만 이용하면 과대적합을 야기할 수 있음

## 1) ['일자']를 기준으로 train.csv + level.csv

In [9]:
train = pd.merge(train, level, left_on = '일자', right_on = '일자', how = 'left')
test = pd.merge(test, level, left_on = '일자', right_on = '일자', how = 'left')

# 8. 훈련 데이터 - Train

## 1) 잘못 기록된 데이터
- 2018-06-01 월 <- 금요일인데 데이터 잘못 기록됨

In [10]:
train.loc[train['일자'] == '2018-06-01', ('요일')] = '금'

## 2) Train ['휴일']

### train 데이터셋은 평일(월화수목금)만 있음. 평일인데 [휴일]은 데이터는 빠져있음
[2016-02-08 월, 2016-02-09 화, 2016-02-10 수, 2016-03-01 화, 2016-04-13 수,
 2016-05-05 목, 2016-05-06 금, 2016-06-06 월, 2016-08-15 월, 2016-09-14 수,
 2016-09-15 목, 2016-09-16 금, 2016-10-03 월, 2017-01-27 금, 2017-01-30 월,
 2017-03-01 수, 2017-05-01 월, 2017-05-03 수, 2017-05-05 금, 2017-05-09 화,
 2017-06-06 화, 2017-08-15 화, 2017-10-09 월, 2017-12-25 월, 2018-01-01 월,
 2018-02-15 목, 2018-02-16 금, 2018-03-01 목, 2018-05-01 화, 2018-05-07 월,
 2018-05-22 화, 2018-06-06 수, 2018-06-13 수, 2018-08-15 수, 2018-10-09 화,
 2018-12-25 화, 2019-01-01 화, 2019-02-04 월, 2019-02-05 화, 2019-02-06 수,
 2019-03-01 금, 2019-05-01 수, 2019-05-06 월, 2019-06-06 목, 2019-08-15 목,
 2019-09-12 목, 2019-09-13 금, 2019-10-01 화, 2019-10-03 목, 2019-10-09 수,
 2019-12-25 수, 2020-01-01 수, 2020-01-24 금, 2020-01-27 월, 2020-04-15 수,
 2020-04-30 목, 2020-05-01 금, 2020-05-05 화, 2020-06-15 월, 2020-06-16 화,
 2020-06-17 수, 2020-06-18 목, 2020-06-19 금, 2020-06-22 월, 2020-06-23 화,
 2020-06-24 수, 2020-06-25 목, 2020-06-26 금, 2020-06-29 월, 2020-06-30 화,
 2020-08-17 월, 2020-09-29 화, 2020-09-30 수, 2020-10-01 목, 2020-10-02 금,
 2020-10-09 금, 2020-12-25 금, 2020-12-28 월, 2020-12-29 화, 2020-12-30 수,
 2020-12-31 목, 2021-01-01 금]

## 3) Train ['휴일전날']

### [휴일] 하루 전 근무일은 석식 이용을 피하고 일찍 퇴근하고자하는 심리를 반영하여 [휴일전날] 생성
[2016-02-05 금, 2016-02-29 월, 2016-04-12 화, 2016-05-04 수, 2016-06-03 금,
 2016-08-12 금, 2016-09-13 화, 2016-09-30 금, 2017-01-26 목, 2017-01-27 금,
 2017-02-28 화, 2017-04-28 금, 2017-05-02 화, 2017-05-04 목, 2017-05-08 월,
 2017-06-05 월, 2017-08-14 월, 2017-10-06 금, 2017-12-22 금, 2018-12-29 금,
 2018-02-14 수, 2018-02-28 수, 2018-04-30 월, 2018-05-04 금, 2018-05-21 월,
 2018-06-05 화, 2018-06-12 화, 2018-08-14 화, 2018-10-08 월, 2018-12-24 월,
 2019-12-31 월, 2019-02-01 금, 2019-02-28 목, 2019-04-30 화, 2019-05-03 금,
 2019-06-05 수, 2019-08-14 수, 2019-09-11 수, 2019-09-30 월, 2019-10-02 수,
 2019-10-08 화, 2019-12-24 화, 2019-12-31 화, 2020-01-23 목, 2020-01-24 금,
 2020-04-24 화, 2020-04-29 수, 2020-05-04 월, 2020-06-12 금, 2020-08-14 금,
 2020-09-28 월, 2020-10-08 목, 2020-12-24 목]

In [11]:
train['휴일전날'] = False

In [12]:
before_holiday = [
    '2016-02-05', '2016-02-29', '2016-04-12', '2016-05-04',
    '2016-06-03', '2016-08-12', '2016-09-13', '2016-09-30',
    '2017-01-26', '2017-01-27', '2017-02-28', '2017-04-28',
    '2017-05-02', '2017-05-04', '2017-05-08', '2017-06-05',
    '2017-08-14', '2017-10-06', '2017-12-22', '2018-12-29',
    '2018-02-14', '2018-02-28', '2018-04-30', '2018-05-04',
    '2018-05-21', '2018-06-05', '2018-06-12', '2018-08-14',
    '2018-10-08', '2018-12-24', '2019-12-31', '2019-02-01',
    '2019-02-28', '2019-04-30', '2019-05-03', '2019-06-05',
    '2019-08-14', '2019-09-11', '2019-09-30', '2019-10-02',
    '2019-10-08', '2019-12-24', '2019-12-31', '2020-01-23',
    '2020-01-24', '2020-04-24', '2020-04-29', '2020-05-04',
    '2020-06-12', '2020-08-14', '2020-09-28', '2020-10-08',
    '2020-12-24'
]
for i in before_holiday:
    train.loc[train['일자'] == i, ('휴일전날')] = True

## 4) Train ['석식메뉴']

### 특정 일자에는 [석식메뉴] 데이터가 음식이 아니고 석식을 하지 않는 이유를 적어놓았음. 이것을 '없음'으로 변환
[2016-11-30 수, 2016-12-28 수, 2017-01-25 수, 2017-02-22 수, 2017-03-22 수,
 2017-04-26 수, 2017-05-31 수, 2017-06-28 수, 2017-07-26 수, 2017-09-01 금,
 2017-09-29 금, 2017-10-25 수, 2017-11-29 수, 2017-12-27 수, 2018-01-31 수,
 2018-02-28 수, 2018-03-28 수, 2018-04-25 수, 2018-05-30 수, 2018-06-22 금,
 2018-06-27 수, 2018-07-25 수, 2018-08-29 수, 2018-09-19 수, 2018-10-31 수,
 2018-11-28 수, 2018-12-26 수, 2019-01-30 수, 2019-02-27 수, 2019-03-27 수,
 2019-04-24 수, 2019-05-29 수, 2019-06-26 수, 2019-07-31 수, 2019-08-28 수,
 2019-09-25 수, 2019-10-30 수, 2019-11-27 수, 2019-12-31 화, 2020-01-29 수,
 2020-02-26 수, 2020-11-25 수]

In [13]:
dinner_null = [
    '2016-11-30', '2016-12-28', '2017-01-25', '2017-02-22',
    '2017-03-22', '2017-04-26', '2017-05-31', '2017-06-28',
    '2017-07-26', '2017-09-01', '2017-09-29', '2017-10-25',
    '2017-11-29', '2017-12-27', '2018-01-31', '2018-02-28',
    '2018-03-28', '2018-04-25', '2018-05-30', '2018-06-22',
    '2018-06-27', '2018-07-25', '2018-08-29', '2018-09-19',
    '2018-10-31', '2018-11-28', '2018-12-26', '2019-01-30',
    '2019-02-27', '2019-03-27', '2019-04-24', '2019-05-29',
    '2019-06-26', '2019-07-31', '2019-08-28', '2019-09-25',
    '2019-10-30', '2019-11-27', '2019-12-31', '2020-01-29',
    '2020-02-26', '2020-11-25'
]
for i in dinner_null:
    train.loc[train['일자'] == i, ('석식메뉴')] = '없음'

# 9. 테스트 데이터 - Test

## 1) Test ['휴일']
[2021-02-11 목, 2021-02-12 금, 2021-03-01 월]

## 2) Test ['휴일전날']
[2021-02-10 수, 2021-02-26 금]

In [14]:
test['휴일전날'] = False

In [15]:
before_holiday = [
    '2021-02-10', '2021-02-26'
]
for i in before_holiday:
    test.loc[test['일자'] == i, ('휴일전날')] = True

# 10. 데이터 공통 전처리 - Train, Test, Submission

## 1) [일자]
- [년], [월], [일]

In [16]:
def ymd(df):
    df['일자'] = pd.to_datetime(df['일자'], format = '%Y-%m-%d')
    df['년'] = df['일자'].dt.year
    df['월'] = df['일자'].dt.month
    df['일'] = df['일자'].dt.day
    # df = df.drop('일자', axis = 1)
    return df

In [17]:
train = ymd(train)
test = ymd(test)
submission = ymd(submission)

## 2) [요일]

In [18]:
train['요일'] = train['요일'].map({'월' : 0, '화' : 1, '수' : 2, '목' : 3, '금' : 4})
test['요일'] = test['요일'].map({'월' : 0, '화' : 1, '수' : 2, '목' : 3, '금' : 4})

## 3) [중식메뉴], [석식메뉴]
- (원산지) 제거
- 토큰화

In [19]:
def menu_embedding(x):
    tmp = []
    x = x.split(' ')
    try:
        for i in x:
            if '(' in i and ':' in i and ')' in i:
                continue
            if '/' in i:
                tmp.extend(i.split('/'))
            else:
                tmp.append(i)
        tmp = list(set(tmp))
        tmp.remove('')
        return tmp
    except:
        return tmp

In [20]:
train['중식메뉴_'] = train['중식메뉴'].apply(lambda x: menu_embedding(x))
train['석식메뉴_'] = train['석식메뉴'].apply(lambda x: menu_embedding(x))

test['중식메뉴_'] = test['중식메뉴'].apply(lambda x: menu_embedding(x))
test['석식메뉴_'] = test['석식메뉴'].apply(lambda x: menu_embedding(x))

## 4) [식사 가능 인원] = [총 인원] - [휴가 인원] - [출장 인원] - [재택 근무 인원]

In [21]:
train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['본사출장자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['본사출장자수'] - test['현본사소속재택근무자수']

## 5) [참여율] = [이용 인원] / [식사 가능 인원]

In [22]:
train['중식참여율'] = train['중식계'] / train['식사가능자수']
train['석식참여율'] = train['석식계'] / train['식사가능자수']

## 6) 필요없는 컬럼 Drop

In [23]:
train = train.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis = 1)
test = test.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis = 1)

## 7) 정수형으로 변환

In [24]:
columns = [
    '본사정원수', '본사휴가자수', '본사출장자수',
    '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
    '중식계', '석식계',
    '식사가능자수', '코로나신규확진자', '휴일전날'
]
for i in columns:
    train[i] = train[i].astype(int)

columns = [
    '본사정원수', '본사휴가자수', '본사출장자수',
    '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
    '식사가능자수', '코로나신규확진자', '휴일전날'
]
for i in columns:
    test[i] = test[i].astype(int)

## 8) 결측치 확인

In [25]:
train.isna().sum()

일자                0
요일                0
본사정원수             0
본사휴가자수            0
본사출장자수            0
본사시간외근무명령서승인건수    0
현본사소속재택근무자수       0
중식계               0
석식계               0
체감온도(중식)          0
불쾌지수(중식)          0
우산(중식)            0
체감온도(석식)          0
불쾌지수(석식)          0
우산(석식)            0
코로나신규확진자          0
거리두기단계            0
휴일전날              0
년                 0
월                 0
일                 0
중식메뉴_             0
석식메뉴_             0
식사가능자수            0
중식참여율             0
석식참여율             0
dtype: int64

In [26]:
test.isna().sum()

일자                0
요일                0
본사정원수             0
본사휴가자수            0
본사출장자수            0
본사시간외근무명령서승인건수    0
현본사소속재택근무자수       0
체감온도(중식)          0
불쾌지수(중식)          0
우산(중식)            0
체감온도(석식)          0
불쾌지수(석식)          0
우산(석식)            0
코로나신규확진자          0
거리두기단계            0
휴일전날              0
년                 0
월                 0
일                 0
중식메뉴_             0
석식메뉴_             0
식사가능자수            0
dtype: int64

# 11. 데이터 저장
- 전처리 완료된 데이터

## 1) 인원 수 예측용 Train 데이터

In [27]:
train_pre = train[[
    '일자', '년', '월', '일', '요일', '휴일전날',
    '본사정원수', '본사휴가자수', '본사출장자수',
    '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
    '식사가능자수',
    '중식계', '석식계',
    '중식참여율', '석식참여율',
    '체감온도(중식)', '불쾌지수(중식)', '우산(중식)',
    '체감온도(석식)', '불쾌지수(석식)', '우산(석식)',
    '코로나신규확진자', '거리두기단계'
]]

In [28]:
train_pre.to_csv('data/train_pre.csv', index = False)

## 2) 인원 수 예측용 Test 데이터

In [29]:
test_pre = test[[
    '일자', '년', '월', '일', '요일', '휴일전날',
    '본사정원수', '본사휴가자수', '본사출장자수',
    '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
    '식사가능자수',
    '체감온도(중식)', '불쾌지수(중식)', '우산(중식)',
    '체감온도(석식)', '불쾌지수(석식)', '우산(석식)',
    '코로나신규확진자', '거리두기단계'
]]

In [30]:
test_pre.to_csv('data/test_pre.csv', index = False)

## 3) 메뉴 분류용 Train 데이터

In [31]:
train.rename(columns = {'중식메뉴_' : '중식메뉴'}, inplace = True)
train.rename(columns = {'석식메뉴_' : '석식메뉴'}, inplace = True)
train_menu = train[[
    '일자', '년', '월', '일', '요일',
    '휴일전날', '중식메뉴', '석식메뉴'
]]

In [32]:
train_menu.to_csv('data/train_menu.csv', index = False)

## 4) 메뉴 분류용 Test 데이터

In [33]:
test.rename(columns = {'중식메뉴_' : '중식메뉴'}, inplace = True)
test.rename(columns = {'석식메뉴_' : '석식메뉴'}, inplace = True)
test_menu = train[[
    '일자', '년', '월', '일', '요일',
    '휴일전날', '중식메뉴', '석식메뉴'
]]

In [34]:
test_menu.to_csv('data/test_menu.csv', index = False)